In [1]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
data = pd.read_csv('data.csv')

#pre-processing to calculate outside good shares
shares = data[['Market_ID','Inside Good Share','PPO']].copy()
shares['PPO Share'] = data['Inside Good Share'] * data['PPO']
shares['HMO Share'] = data['Inside Good Share'] * (1 - data['PPO'])

group_shares = shares.groupby('Market_ID').sum()
group_shares['Outside Good Share'] = 1 - group_shares['Inside Good Share']

data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')

print data.min()

Market_ID             1.000000
Plan_ID               1.000000
PPO                   0.000000
Network Score         0.780000
Satisfaction Score    0.725000
Premium               2.269728
Inside Good Share     0.037670
Outside Good Share    0.096070
dtype: float64


In [3]:
#first estimate using logit
class logit(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(logit, self).__init__(*args, **kwds)

        
    def momcond(self, params):
        #unwrap stuff
        shares = np.array(self.endog).transpose()
        exog = np.array(self.exog)
        instr = np.array(self.instrument)
        
        lshare = np.log(shares[0]) -  np.log(shares[1])
        lshare = lshare.transpose()
       
        lshare_fit = np.matmul(exog,params) #linear equation    
        
        xi = lshare_fit - lshare
        g = instr * xi[:, np.newaxis]
        
        return g

In [4]:
#calculate hausmann insturments
mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
hausman_instr = join(plan_dum

#set up x and y
y = data[['Inside Good Share','Outside Good Share']]
x =  data[['Network Score','Satisfaction Score','PPO','Premium']]

In [5]:
#set up initial est
beta_init = np.full(len(x.columns),1)

#set up model
model = logit(y , x, hausman_instr)

result = model.fit(beta_init, maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 292
         Function evaluations: 497
Optimization terminated successfully.
         Current function value: 0.005149
         Iterations: 139
         Function evaluations: 249
                                             logit Results                                             
Dep. Variable:     ['Inside Good Share', 'Outside Good Share']   Hansen J:                        16.99
Model:                                                   logit   Prob (Hansen J):                 0.108
Method:                                                    GMM                                         
Date:                                         Wed, 10 Oct 2018                                         
Time:                                                 13:32:44                                         
No. Observations:                                         3300                                  

In [26]:
#correct way to compute shares...

def comp_shares(x,p,alpha,beta,labels=False):
    #compute exp(delta_j)
    x = x.copy()
    x['exp_delta'] =  np.exp ( np.matmul(x[['Network Score','Satisfaction Score','PPO']],beta) - alpha*p )
    #print x['exp_delta']
    
    #compute 1 + sum_j exp(delta_j)
    sum_delta = x.groupby('Market_ID').sum()
    sum_delta['sum_exp_delta'] = 1 + sum_delta['exp_delta'] 
    
    x = pd.merge(x, sum_delta[['sum_exp_delta']], 
                right_index=True, left_on = 'Market_ID')

    #compute s_j
    x['fitted_share'] = x['exp_delta']/x['sum_exp_delta']
    
    if labels: 
        return x[['Plan_ID','Market_ID','fitted_share']]
    
    return np.array(x['fitted_share']).squeeze()
        

#set up parameters 
observ = data[['Plan_ID','Market_ID','Network Score','Satisfaction Score','PPO']]
prices = np.array( data['Premium'] ) #needed for later...

beta = result.params[:-1]
alpha = abs(result.params[3])

shares = comp_shares(observ,prices,alpha,beta)

print shares

[0.17502021 0.23839869 0.29850248 ... 0.09512126 0.09887772 0.13083635]


In [16]:
def comp_elasticity(shares, prices, alpha):
    #set up matrix skeleton
    own_price = np.identity(len(shares))
    cross_price = 1 - own_price

    #actually calculate elasticity
    cross_elasticity = shares * alpha * prices
    own_elasticity  = -(1-shares) * alpha * prices

    return cross_price*cross_elasticity + own_price *own_elasticity


#agregate elasticities
elasticity = comp_elasticity(shares, prices, alpha)
print elasticity[1][1],elasticity[2][1],elasticity[3][1] #second index tells you wrt what good i.e. denom


#average elasticity
avg_shares =  np.array( comp_shares(observ,prices,alpha,beta,
                                    labels=True).groupby('Plan_ID').mean()['fitted_share']).squeeze()
avg_price = np.array( data[['Plan_ID','Premium']].groupby('Plan_ID').mean() ).squeeze()
avg_elasticity = comp_elasticity(avg_shares, avg_price, alpha)
print avg_elasticity

-3.8405528281901504 1.2021812009896957 1.2021812009896957
[[-4.32655865  0.5554553   1.08916071  0.82684009  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942 -4.3085966   1.08916071  0.82684009  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942  0.5554553  -4.2870623   0.82684009  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942  0.5554553   1.08916071 -4.37731375  0.5712068   0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.00264945
   0.76680516  0.67056723  1.07787354  0.67651484]
 [ 1.01565942  0.5554553   1.08916071  0.82684009 -4.25288515  0.71449052
   1.00282018  0.7442054   0.83712214  0.45917062  0.59919906  1.0026494

In [22]:
#solve for marginal costs
def comp_markup(shares):
    shares_vector = np.array([shares])
    
    #set up matrix    
    own_price = np.identity(len(shares))
    cross_price = 1 - own_price
    #caclulate formula
    cross_deriv = alpha * np.matmul(shares_vector.transpose() , shares_vector )
    own_deriv  = - alpha * (1-shares)  * shares
    
    derivative =  cross_price*cross_deriv + own_price *own_deriv
    #take inverse and calc markup
    inv_derivative = np.linalg.inv(derivative)

    markup = np.matmul(inv_derivative, shares_vector.transpose()) 
    return markup.transpose()[0]


markup = comp_markup(shares)
mc = prices - markup
print markup
print mc

[0.00096085 0.00096085 0.00096085 ... 0.00096085 0.00096085 0.00096085]
[2.49680175 2.44522855 2.79422575 ... 2.66357285 2.35594355 2.52566855]


In [23]:
def profits(p, rebate=.25, x=observ, beta=beta):
    shares = comp_shares(x, p, alpha, beta)
    return - np.matmul(shares, np.array([prices-mc]).transpose())

print profits(prices, rebate = 0)

[-0.48605274]


In [25]:
res = minimize(profits, prices, method='nelder-mead',
     options={'xtol': 1e-4, 'disp': True})

print(res.x) #doesn't seem right

KeyboardInterrupt: 

In [ ]:
#profits per enrollee
#outside good shares


In [ ]:
#consumer surplus
#1/alpha *  ( sum(np.exp(delta1_j)) - sum(np.exp(delta0_j)) )